In [1]:
import torch
import torch.nn as nn
from sinabs.backend.dynapcnn import DynapcnnNetwork
from sinabs.layers import Merge, IAFSqueeze, SumPool2d
from sinabs.activation.surrogate_gradient_fn import PeriodicExponential

In [2]:
torch.manual_seed(0)

In [3]:
channels = 2
height = 34
width = 34
batch_size = 8

input_shape = (channels, height, width)

## Network Module

We need to define a `nn.Module` implementing the network we want the chip to reproduce.

In [4]:
class SNN(nn.Module):
    def __init__(self) -> None:
        super().__init__()

        self.conv1 = nn.Conv2d(2, 10, 2, 1, bias=False) # node 0
        self.iaf1 = IAFSqueeze(batch_size=batch_size, min_v_mem=-1.0, spike_threshold=1.0, surrogate_grad_fn=PeriodicExponential())            # node 1
        self.pool1 = nn.AvgPool2d(3,3)                  # node 2
        self.pool1a = nn.AvgPool2d(4,4)                 # node 3

        self.conv2 = nn.Conv2d(10, 10, 4, 1, bias=False)# node 4
        self.iaf2 = IAFSqueeze(batch_size=batch_size, min_v_mem=-1.0, spike_threshold=1.0, surrogate_grad_fn=PeriodicExponential())            # node 6

        self.conv3 = nn.Conv2d(10, 1, 2, 1, bias=False) # node 8
        self.iaf3 = IAFSqueeze(batch_size=batch_size, min_v_mem=-1.0, spike_threshold=1.0, surrogate_grad_fn=PeriodicExponential())            # node 9

        self.flat = nn.Flatten()

        self.fc1 = nn.Linear(49, 500, bias=False)       # node 10
        self.iaf4 = IAFSqueeze(batch_size=batch_size, min_v_mem=-1.0, spike_threshold=1.0, surrogate_grad_fn=PeriodicExponential())            # node 11
        
        self.fc2 = nn.Linear(500, 10, bias=False)       # node 12
        self.iaf5 = IAFSqueeze(batch_size=batch_size, min_v_mem=-1.0, spike_threshold=1.0, surrogate_grad_fn=PeriodicExponential())            # node 13

        self.adder = Merge()

    def forward(self, x):
        
        con1_out = self.conv1(x)
        iaf1_out = self.iaf1(con1_out)
        pool1_out = self.pool1(iaf1_out)
        pool1a_out = self.pool1a(iaf1_out)

        conv2_out = self.conv2(pool1_out)
        iaf2_out = self.iaf2(conv2_out)

        conv3_out = self.conv3(self.adder(pool1a_out, iaf2_out))
        iaf3_out = self.iaf3(conv3_out)

        flat_out = self.flat(iaf3_out)
        
        fc1_out = self.fc1(flat_out)
        iaf4_out = self.iaf4(fc1_out)
        fc2_out = self.fc2(iaf4_out)
        iaf5_out = self.iaf5(fc2_out)

        return iaf5_out

In [5]:
snn = SNN()

## DynapcnnNetwork Class

In the constructor of `DynapcnnNetworkGraph` the SNN passed as argument (defined as a `nn.Module`) will be parsed such that each layer is represented in a computational graph (using `nirtorch.extract_torch_graph`). 

The layers are the `nodes` of the graph, while their connectivity (how the outputs from a layer are sent to other layers) is represented as `edges`, represented in a `list` of `tuples`.

Once the constructor finishes its initialization, the `hw_model.dynapcnn_layers` property is a dictionary where each entry represents the ID of a `DynapcnnLayer` instance (an `int` from `0` to `L`), with this entry containing a `DynapcnnLayer` instance where a subset of the layers in the original SNN has been incorporated into, the core such instance has been assigned to, and the list of `DynapcnnLayer` instances (their IDs) the layer targets.

In [6]:
hw_model = DynapcnnNetwork(
    snn=snn,
    input_shape=input_shape,
    batch_size=batch_size,
    discretize=False
)

Notice in the model bellow how the property <code>DynapcnnLayer</code> in the model has yet to be assigned to a core. This is only done once
<code>DynapcnnNetworkGraph.to()</code> is called.

In [7]:
print(hw_model)

----------------------- [ DynapcnnLayer 0 ] -----------------------

COMPUTATIONAL NODES:

(node 0): Conv2d(2, 10, kernel_size=(2, 2), stride=(1, 1), bias=False)
(node 1): IAFSqueeze(spike_threshold=Parameter containing:
tensor(1.), min_v_mem=Parameter containing:
tensor(-1.), batch_size=8, num_timesteps=-1)
(node 2): SumPool2d(norm_type=1, kernel_size=(3, 3), stride=None, ceil_mode=False)
(node 3): SumPool2d(norm_type=1, kernel_size=(4, 4), stride=None, ceil_mode=False)

METADATA:

> network's entry point: True
> convolution's weight re-scaling factor: None
> assigned core index: None
> destination DynapcnnLayers: [1, 2]
> node 2 feeds input to nodes [4]
> node 3 feeds input to nodes [7]

----------------------- [ DynapcnnLayer 1 ] -----------------------

COMPUTATIONAL NODES:

(node 4): Conv2d(10, 10, kernel_size=(4, 4), stride=(1, 1), bias=False)
(node 6): IAFSqueeze(spike_threshold=Parameter containing:
tensor(1.), min_v_mem=Parameter containing:
tensor(-1.), batch_size=8, num_time

The `hw_model.to()` call will figure out into which core each `DynapcnnLayer` instance will be assigned to. Once this assingment is made the instance itself is used to configure the `CNNLayerConfig` instance representing the core's configuration assigned to it.

If the call is sucessfull, the layers comprising the network and their associated metadata will be printed.

In [8]:
hw_model.to(device="speck2fmodule:0")

Network is valid: 

----------------------- [ DynapcnnLayer 0 ] -----------------------

COMPUTATIONAL NODES:

(node 0): Conv2d(2, 10, kernel_size=(2, 2), stride=(1, 1), bias=False)
(node 1): IAFSqueeze(spike_threshold=Parameter containing:
tensor(1.), min_v_mem=Parameter containing:
tensor(-1.), batch_size=8, num_timesteps=-1)
(node 2): SumPool2d(norm_type=1, kernel_size=(3, 3), stride=None, ceil_mode=False)
(node 3): SumPool2d(norm_type=1, kernel_size=(4, 4), stride=None, ceil_mode=False)

METADATA:

> network's entry point: True
> convolution's weight re-scaling factor: None
> assigned core index: 0
> destination DynapcnnLayers: [1, 2]
> node 2 feeds input to nodes [4]
> node 3 feeds input to nodes [7]

----------------------- [ DynapcnnLayer 1 ] -----------------------

COMPUTATIONAL NODES:

(node 4): Conv2d(10, 10, kernel_size=(4, 4), stride=(1, 1), bias=False)
(node 6): IAFSqueeze(spike_threshold=Parameter containing:
tensor(1.), min_v_mem=Parameter containing:
tensor(-1.), batch

Notice above now how the layers of the model have been assigned to a chip core.

# Training the HW model

In [9]:
hw_model.init_weights()

In [10]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print('device: ', torch.cuda.get_device_name(0))
else:
    device = torch.device('cpu')

device:  NVIDIA GeForce RTX 3070 Ti


In [11]:
hw_model.to(device)

In [12]:
import sys

sys.path.append('../utils')

from train_test_fn import training_loop, load_dataset, split_train_validation, load_architecture

In [13]:
batch_size = 8
num_workers = 4
epochs = 5
lr = 5e-4

n_time_steps = 50

In [14]:
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

In [15]:
snn_train_dataset, snn_test_dataset, sensor_size, nb_classes = load_dataset('NMNIST', n_time_steps, "../NMNIST")

print(sensor_size)

snn_train_dataloader = DataLoader(snn_train_dataset, batch_size=batch_size, num_workers=num_workers, drop_last=True, shuffle=True)
snn_test_dataloader = DataLoader(snn_test_dataset, batch_size=batch_size, num_workers=num_workers, drop_last=True, shuffle=False)

(34, 34, 2)


In [16]:
optimizer = Adam(hw_model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-8)
loss_fn = CrossEntropyLoss()

In [17]:
epochs_x, epochs_y, epochs_acc = training_loop(
    device, 
    n_time_steps,
    batch_size,
    sensor_size,
    snn_train_dataloader, 
    hw_model, 
    loss_fn, 
    optimizer, 
    epochs, 
    snn_test_dataloader)

  0%|          | 0/7500 [00:00<?, ?it/s]

  0%|          | 0/7500 [00:00<?, ?it/s]

KeyboardInterrupt: 